# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

## 使用微调前 ChatGLM3

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

2024-02-07 17:58:49.492226: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 17:58:49.492250: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 17:58:49.492263: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-07 17:58:49.496154: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-07 17:58:49.990742: W tensorflow/compiler/

In [7]:
print(response)

乾卦是《易经》中的一个卦象，由六个阳爻（表示阳）和一条阴爻（表示阴）组成。乾卦象征着天、云、雷等自然现象，以及君主、领导、力量、刚毅等含义。在《易经》中，乾卦具有很高的地位，被认为是一种至高无上的象征。

乾卦的卦辞（即卦象的说明）如下：

乾：元、亨、利、贞。

这个卦辞分为四个字，分别表示：

1. 元：表示开始、原始，代表万事万物的起源。
2. 亨：表示通、顺利，表示事物发展顺利。
3. 利：表示有益、有利，表示事物有积极的意义。
4. 贞：表示正、正确，表示事物的运行和发展是正当的、正确的。

综合起来，乾卦象征着宇宙万物的起源和发展，以及人类社会的领导、力量和刚毅。


In [8]:
response, history = base_model.chat(tokenizer, query="地水师卦是什么？", history=history)
#response, history = base_model.chat(tokenizer, query="请介绍上海蓝盟网络技术有限公司的创始人？", history=history)
print(response)

地水师卦是《易经》中的一个卦象，由两个阴爻和五个阳爻组成。这个卦象象征着水，以及军队、众志成城、众望所归等含义。在《易经》中，地水师卦的位置位于北方，与事业、努力、艰难、团结等有关。

地水师卦的卦辞（即卦象的说明）如下：

地水师：贞元，志。

这个卦辞分为两个字，分别是：

1. 贞元：表示正、正确，表示事物的运行和发展是正当的、正确的。
2. 志：表示决心、志向，表示人们为了实现某种目标而坚定不移地努力。

综合起来，地水师卦象征着人们在水资源方面的发展和努力，以及在这个过程中所表现出的坚定决心和志向。


## 微调前后效果对比

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

### 使用微调后的 ChatGLM3-6B

In [10]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [11]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，它由六个阳爻（代表刚健的特性）排列而成，象征着刚健、积极、强盛的意象。在卜问中，乾卦预示着刚强、顺利的形势，并且 indicates应该刚健自信，勇于面对挑战。在事业、财运、治理国家等方面，乾卦都象征着能力和威仪，需要保持谦逊、刚强、勇于担当的态度。

乾卦的卦辞是：“元、亨、利、贞”，其中“元”表示 beginning（起始），"亨"表示 path（道路），"利"表示 benefit（利益），"贞"表示正（正派）."

乾卦的哲学内涵是强调刚强、积极、勇于担当的品质。在卜问中，表示刚强、顺利的形势，并且 indicates应该刚健自信，勇于面对挑战。在事业、财运、治理国家等方面，乾卦都象征着能力和威仪，需要保持谦逊、刚强、勇于担当的态度。

乾卦的刚强、自信和勇于担当的特质，使得它成为预测中的重要元素。在卜问中，乾卦预示着刚强、顺利的形势，并且 indicates应该刚健自信，勇于面对挑战。在事业、财运、治理国家等方面，乾卦都象征着能力和威仪，需要保持谦逊、刚强、勇于担当的态度。


ChatGLM3-6B微调后：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，具有刚健强劲的特性。它由六个阳爻（代表阳）组成，排列成两个阳爻夹一个阴爻的格式。乾卦的卦辞为：“元、亨、利、贞”，意味着天是元始的，亨通无阻，利于正道，忠诚于国。乾卦的时运为：天运贞正，国运光明。

乾卦的核心哲学是：天是刚健强劲的，充满了活力和生机，因此我们应当效法天的这些特性，努力奋斗，寻求光明的前途。在运势方面，顺境时宜谨慎，逆境时宜坚持，能够digest顺境的繁荣，能够忍受逆境的艰辛。

在占卜中，乾卦表示着事态的刚性和变化，以及人们对此的期望和祝愿。它预示着事业上的成功和顺利，同时也预示着必须努力奋斗，以保持诚实、正直和忠诚的品质。

在决策上，适合采用刚强坚毅的人，并且要注重策略和规划，努力实现目标。


In [12]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是一个由《易经》中的卦象组成，由上卦坤（地）和下卦坎（水）组合而成。在卜筮时，上卦坤表示地，下卦坎表示水，代表大地满满的，象坎一样流动。

地水师卦的卦象特点是上卦为坤，下卦为坎。上卦坤代表地，下卦坎代表水，两者相辅相成，象征着地中的水流动，预示着大吉大利。

在《象辞》中，地水师卦被描述为：地中有水，地中有甘美的水流。

在占卜中，地水师卦表示着大吉大利的卦象。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是中国传统易学中的一卦，由上卦坤（地）和下卦坎（水）组成，上卦为坤地，下卦为坎水。地水相克，预示着占卜者将面临困难，需要等待时机的到来。

在《易经》中，卦辞这样描述地水师卦：

"地水师卦：初爻为坤，二爻为坎，三爻为离。离为火，坤为地，地水相克。此卦数为未，象为火，预示着困难 will 到来，未为火，卦数为未，因此这个卦象预示着好运。

在塔山之游卦中，地水师卦表示地水相克，需要等待时机。在占卜中，初始阶段可能会遇到困难和挑战，需要保持冷静，坚持信念，等待时机，仍未可知。

在命运学中，地水师卦意味着困难将会到来，需要等待时机。在决策过程中，应该考虑到当前的形势和环境，以及可能出现的变数，以做出正确的选择。

在解卦中，地水师卦表示困难 will 到来，未为火，卦数为未，因此这个卦象预示着好运。

总结起来，地水师卦预示着困难将会到来，需要等待时机。在决策过程中，应该考虑到当前的形势和环境，以及可能出现的变数，以做出正确的选择。


In [16]:
base_response, ft_response = compare_chatglm_results(query="天水讼卦")

问题：天水讼卦

原始输出：
讼卦是周易卦象中的一枚卦，由上卦坤（地）和下卦乾（天）组成，预示着天象和地象之间的矛盾和冲突。在讼卦中，天（乾）代表刚强，地（坤）代表柔顺。在当前的形势下，刚强的天际主客观求解决已经存在的矛盾和问题，但柔顺的地暂避锋芒，待时而行。

讼卦的核心观念是：刚强的物易引起矛盾和冲突，因此需要柔顺和包容来化解矛盾。在决策过程中，需要谨慎对待形势，避免冲突升级，以保持和平和稳定的态度。

讼卦的哲学意义取决于卦象的组合和解释，这需要结合具体的情境和状况来分析。在实际应用中，讼卦通常被认为是一种积极的象征，意味着有力量和决心去解决问题和冲突。然而，刚强和冲突可能会导致不必要的损失，因此需要保持冷静和理智，以避免冲突升级。


ChatGLM3-6B微调后：
[gMASK]sop 天水讼卦


的天水讼卦是一个由坎卦（水）和卦（上卦为坎，下卦为卦）组成的卦象，代表着水在地下流动，象征着 underground water flow. This卦 is associated with the element water and is associated with the concept of water flowing underground, which symbolizes the idea of justice and fairness.

In the traditional Chinese culture, the天水讼卦 is considered to be a warning sign of an impending disaster, and it is recommended to take necessary precautions to avert the worst. This divination symbolizes the importance ofmodifying one's ways and considering the consequences of one's actions, and it serves as a reminder of the need for self-improvement and self-reflection.

In the traditional Chinese divina

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [15]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [16]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，具有刚健强劲的特性。它由两个乾卦叠加而成，象征着天地之间的相互支撑和互相照耀。在周易中，乾卦代表天，反映刚健强劲的特性。同时，乾卦也代表君子，具有刚健强劲的品德，以及天命般的运行规律。

乾卦的卦辞是：“元、亨、利、贞”，意味着元（ start）亨（顺利）利（吉祥）贞（正），象征着事物天命般的运行规律。同时，乾卦也表达了敬天保元、顺从天意的寓意。

在周易中，乾卦具有以下特性和意义：

1. 象征天：乾卦代表天，象征着刚健强劲的特性。

2. 刚健强劲：乾卦由两个乾卦叠加而成，象征着天地之间的相互支撑和互相照耀。反映了刚健强劲的特性。

3. 天地相互支援：在周易中，乾卦代表天，反映了天命般的运行规律。

4. 敬天保元：乾卦的卦辞中提到“元、亨、利、贞”，表达了敬天保元、顺从天意的寓意。

5. 吉祥：乾卦象征着吉祥如意，代表着天命上的正统和合法。

6. 刚健生动：乾卦具有刚健生动的特质，代表着君子刚健的品德。

7. 刚直不阿：乾卦反映刚健强劲的品德，象征着君子的刚直不阿。

8. 天地之间：在周易中，乾卦代表天，象征着君子应该效法天的运行规律，努力刚健刚直，实现天命般的运行。

9. 刚健强劲：乾卦的特性是刚健强劲，代表着不阿不屈不妥协的品质。

总结起来，乾卦是周易中的一卦，代表天，具有刚健强劲的特性。它象征着天地之间的相互支撑和互相照耀，表达了敬天保元、顺从天意的寓意。同时，乾卦也代表君子，具有刚健强劲的品德，以及天命般的运行规律。


ChatGLM3-6B微调后：
[gMASK]sop 解释下乾卦是什么？ 
乾卦是周易中的一卦，代表天，具有刚健强劲的特性。它由两个卦相组合而成，一个是乾卦，另一个是坤卦。乾卦的排列是：元、亨、利、贞，代表天象运行，具有刚健强劲的特性。

在周易中，乾卦预示着天将健行， Cloud Deck（云层）预示着天将有所得，丽（美丽）预示着天将利市，利（吉祥）预示着天将得利。

乾卦的哲学内涵是：天行健，君子以自强不息。它强调的是刚健强劲的品质，鼓励人们不断自我提高，自我完善。

在周易中，乾卦代表的是天，具有刚健强劲的特性。它预示着天将健行，云层将有所得，丽将利市，利将得利。

在周易的哲学中，乾卦预示着天将健行， Cloud Deck（云层）预示着天将有所得

In [17]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是周易中的一卦，由两个卦象组合而成：上卦为坤，代表大地，代表顺从、顺应天道的人；下卦为坎，代表水，代表变化、灵活。

地水师卦的核心卦象是“地水相成”，代表水在地下流动，大地吸收水分，象征着顺利融合双方的力量，代表着顺利和谐地处理双方的矛盾。

在占卜时，地水师卦预示着会遇到困难，必须 flexibility（灵活），求变，代表变通，求变则通。代表预测：困难已至，宜速变通，宜谨慎行事。

地水师卦的核心哲学是：调料需调和使用， wealth（财富）需谨慎使用，二者必需相互配合。

地水师卦适用于占卜、预测未来的情况，提醒人们遇到困难和变故，必须谨慎处理，变通求通。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由坎卦（水）和坤卦（地）组成的卦象，代表地下的水，象征润泽大地的力量。根据《易经》中的解释，地水师卦预示着吉祥如意，具有强大的力量，可以克服各种困难。

根据传统解卦，地水师卦分为两个卦象，第一个卦象是坎卦（水），代表灵活变化的力量，具有吉祥如意，靠智慧取悦于人；第二个卦象是坤卦（地），代表刚健柔顺，具有强大的力量，可以克服各种困难。

地水师卦的核心理念是：吉祥如意，靠智慧取悦于人，靠刚健柔顺的力量，可以克服各种困难。


In [19]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
讼卦是周易卦象中的一枚卦，由上卦坤（地）和下卦乾（天）组成，象征着天地的相争，具有象征意义。在卜筮时，先提出天象，再详观地象，以确定双方的态度，从而决定是否彼此相争。同时，对方是否愿意主动求胜？在决策时，要考虑利弊得失，同时要考虑双方的意愿，双方都希望得到想要的东西。

讼卦的核心哲学是：天地的相争，双方都需要谨慎处理，通过协商和智慧，以达成共识。在实际应用中，讼卦提醒我们在面对争端和冲突时，要冷静分析，以ilegitimous（合理）的方式解决争端，以避免冲突进一步升级。


ChatGLM3-6B微调后：
[gMASK]sop 周易中的讼卦是什么卦象

 讼卦是周易卦象中的一卦，由上卦坎（水）和下卦乾（天）组成，象征着天神下降于坎水之中，具有刚健刚毅之象。讼卦的卦象具有云雾聚集、天光破云的意象，象征着事情将会顺利发展，宜早做准备。

讼卦的时运是：
 初爻：阳爻（天）刚健刚毅
 二爻：阳爻（天）刚健刚毅
 三爻：阳爻（天）刚健刚毅
 四爻：阳爻（天）刚健刚毅
 初爻：阳爻（天）刚健刚毅
 限运：
 阳爻（天）刚健刚毅
 阴爻（天）刚柔温和
 阳爻（天）刚健刚毅
 阳爻（天）刚健刚毅

讼卦的解卦是：
 初爻：阳爻（天）刚健刚毅
 二爻：阳爻（天）刚健刚毅
 三爻：阳爻（天）刚健刚毅
 初爻：阳爻（天）刚健刚毅
 限运：
 阳爻（天）刚健刚毅
 阴爻（天）刚柔温和
 阳爻（天）刚健刚毅
 阳爻（天）刚健刚毅

讼卦的卦象和时运都预示着刚健刚毅的特性，天神下降于坎水之中，具有刚健刚毅之象。在事业和生活中遇到困难和挑战时，需要积极应对，宜早做准备，不宜冒险，切勿冒进。在处理问题和决策时，要冷静分析，不宜草率行事，宜等待时机的到来。同时，要注重刚健刚毅的态度，积极应对各种困难和挑战，从而获得成功。


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed))

In [13]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20240208_003656"
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [14]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：\n{ft_response}")
    return base_response, ft_response

In [15]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是八卦之一，也是八宫图之一，其卦象是由三个阳爻夹一个阴爻构成，象征着天、强、积极、刚毅等含义。乾卦的卦辞为“元、亨、利、贞”，表示这个卦具有正统、顺利、有益和正定的意义。在易经中，乾卦代表一种积极向上的精神，是一种推动、领导、创造和发展的力量。乾卦也象征着父亲、领袖、君主等权力和威严，同时也代表着刚强、坚定和果敢等品质。在股市中，乾卦也代表着市场上涨、股票上涨等积极的投资意义。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 解释下乾卦是什么？
乾卦是八卦之一，它的卦象是由三个阳爻夹一个阴爻构成，象征着天、云、雷等自然现象，也代表着刚强、积极、行动力等含义。在八宫图（后天八卦）中，乾卦位于北方，与事业、努力、坚定等有关。在五行中，乾卦对应于木，代表着生长、茂盛、积极向上等含义。乾卦的意义在于鼓励人们保持积极向上的心态，勇于担当，坚定信念，努力奋斗，实现自己的目标。


In [16]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
“地水师卦”是中国古代的一种卜卦方法,也称为“山水师卦”、“地水卦”等。它是《易经》中的一种卦象,由两个卦相组合而成,分别是“地卦”和“水卦”。

“地卦”代表“山”和“土”,象征着坚定、稳定、静止、包容等特质;“水卦”则代表“水”和“润”,象征着流动、变化、柔软、流动等特质。

当这两个卦象组合在一起时,它们形成了一个具有丰富象征意义的卦象,代表着一种平衡、和谐、合作的状态。具体来说,它意味着:

- 坚持原有稳定的状态,不要轻言改变;
- 接受变化和流动,保持开放的心态;
- 注重协调和平衡,不要过于偏激;
- 关注人与自然的关系,保持和谐的状态。

“地水师卦”是一种以自然现象为象征的卜卦方法,旨在帮助人们更好地理解自然界的规律,以及人与自然的关系,从而指导人们做出正确的决策。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 地水师卦是什么？
地水师卦是易经中的一个卦象，由两个卦相组合而成：地卦和水分卦。地卦代表土地，象征稳定、坚守、守旧；水分卦代表水，象征变化、流动、流动。将这两个卦相组合起来，形成地水师卦，寓意着土地和水的相互影响，以及坚守与变化的平衡。

地水师卦的意义主要体现在以下几个方面：

1. 土地与水的关系：土地是承载水的基础，水是滋润土地的源泉。在这个卦象中，土地和水的相互影响和相互依存关系被强调。

2. 坚守与变化的平衡：地水师卦提醒我们在处理问题和决策时要坚守基本原则，但同时也要具备灵活变通的能力，因为只有坚守与变化相结合，才能在事物发展过程中取得成功。

3. 团结协作：地水师卦还强调了团结协作的重要性。土地和水分离不开彼此，同样，人在处理问题时也需要各种资源和力量的相互支持，只有团结协作，才能共同克服困难，取得成功。

总之，地水师卦是易经中的一种卦象，它体现了土地与水、坚守与变化、团结协作等重要观念，对于我们在生活和工作中遇到的问题，有一定的指导和启示作用。


In [17]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
周易中的讼卦（后天八卦之一，对应于乾卦）表示诉讼、争端、诉讼纠纷等。它是由两个卦相组合而成的：乾为天，代表刚强、刚毅、正义等品质；坤为地，代表顺从、温和、包容等品质。讼卦象征天与地之间的矛盾和冲突，表示诉讼和争端可能会因为双方坚持己见而加剧。

在周易中，讼卦的解读需要结合其他卦来分析。通常情况下，讼卦的出现提示要避免争端，以和为贵。具体如何化解诉讼和争端，需要根据卦象、爻辞和 other 八卦来综合判断。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 周易中的讼卦是什么？
周易中的讼卦是八卦之一，它的卦象是由两个离字叠加而成，表示诉讼、争端、诉讼等概念。讼卦的意义是解决纷争，促进和谐。在周易中，它是hexagram（六爻卦）之一，由两个离字叠加而成，表示诉讼、争端、诉讼等概念。
